In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn import svm

In [ ]:
df = pd.read_csv("analysis/not_uploaded/IA/TID/.scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-9.pth.txt")
mos = pd.read_csv("analysis/not_uploaded/mos_with_names.txt", sep=" ")

#df = df[df["img"].str.contains("_")]
df["img"] = df["img"].apply(lambda row: Path(row).stem)
df["img"] = df["img"].apply(lambda row: row.lower())
df["img"] = df["img"].apply(lambda row: row.split(".")[0])
df["img"] = df["img"].apply(lambda row: row if "_" in row else row+"_00_0")

mos["img"] = mos["img"].apply(lambda row: row.lower())
mos["img"] = mos["img"].apply(lambda row: row.split(".")[0])
mos["img"] = mos["img"].apply(lambda row: row if "_" in row else row+"_00_0")


df = df.set_index("img").join(mos.set_index("img"))
df = pd.DataFrame(df.to_records())

df["img_id"] = df["img"].apply(lambda row: int(row.split("_")[0][1:]))
df["distortion"] = df["img"].apply(lambda row: int(row.split("_")[1]))
df["dist_level"] = df["img"].apply(lambda row: int(row.split("_")[2]))

try:
    df["score"] = (df["styles_score"] + df["technical_score"] + df["composition_score"])/3
except:
    pass
df

In [ ]:
def histogram_distortion(distortion:int, score:str):
    plot_frame = df[(df["distortion"] == distortion) | (df["distortion"] == 0)]
    for dist_level in range(6):
        sns.distplot(plot_frame[(plot_frame["dist_level"] == dist_level)][score], label=f"{distortion}: {dist_level}")
    plt.legend()

In [ ]:
def violin_distortion(distortion:int, score:str):
    plot_frame = df[(df["distortion"] == distortion) | (df["distortion"] == 0)]
    sns.violinplot(data=plot_frame, x="dist_level", y=score, color="steelblue")
    plt.legend()

In [ ]:
histogram_distortion(10, "score")

In [ ]:
histogram_distortion(10, "MOSscore")

In [ ]:
violin_distortion(10, "score")

In [ ]:
violin_distortion(10, "MOSscore")

In [ ]:
sns.kdeplot(x=df["MOSscore"], y=df["score"])

In [ ]:
for dist in range(1, 25):
    results = []
    for img in range(1,26):
        results.append(stats.spearmanr(df[(df["img_id"] == img) & (df["distortion"] == dist)]["dist_level"], df[(df["img_id"] == img) & (df["distortion"] == dist)]["score"]))
    print(dist,"\t", sum([val[0] for val in results])/len(results))

In [ ]:
for dist in range(1, 25):
    results = []
    for img in range(1,26):
        results.append(stats.spearmanr(df[(df["img_id"] == img) & (df["distortion"] == dist)]["dist_level"], df[(df["img_id"] == img) & (df["distortion"] == dist)]["MOSscore"]))
    print(dist,"\t", sum([val[0] for val in results])/len(results))

In [ ]:
ranks = []
for dist in range(1, 25):
    results = []
    for img in range(1,26):
        results.append(stats.spearmanr(df[(df["img_id"] == img) & (df["distortion"] == dist)]["MOSscore"], df[(df["img_id"] == img) & (df["distortion"] == dist)]["score"]))
    print(dist,"\t", sum([val[0] for val in results])/len(results))
    ranks.append(sum([val[0] for val in results])/len(results))
print("avg", "\t", sum(ranks) / len(ranks))

In [ ]:
stats.spearmanr(df[["MOSscore", "score"]].dropna())

In [ ]:
features = [c for c in df.columns if c not in ["index", "img", "img_id", "distortion", "dist_level"]]
reg_df = df.dropna()
train = reg_df[reg_df["img_id"] <= 15].reset_index()[features]
test = reg_df[reg_df["img_id"] > 15].reset_index()[features]

In [ ]:
reg = svm.NuSVR().fit(train.drop(columns=["MOSscore"]), train["MOSscore"])
reg.score(train.drop(columns=["MOSscore"]), train["MOSscore"])

In [ ]:
reg.score(test.drop(columns=["MOSscore"]), test["MOSscore"])

In [ ]:
reg.predict(test.drop(columns=["MOSscore"]))

In [ ]:
stats.pearsonr(reg.predict(test.drop(columns=["MOSscore"])), test["MOSscore"])

In [ ]:
sns.kdeplot(x=test["MOSscore"], y=reg.predict(test.drop(columns=["MOSscore"])))

In [ ]:
df[~df["img"].str.contains("00_0")].sort_values(by="img").reset_index()[["score","img"]].to_csv("analysis/not_uploaded/IA/TID/pred_scores", index=False, sep=" ")